In [3]:
from zumi.util.camera import Camera
from zumi.zumi import Zumi

import cv2
import PIL.Image
import IPython.display
import time
import math
import pyzbar.pyzbar as pyzbar

import numpy as np

width = 160 
height = 128

try:
    camera = Camera(width,height)
    camera.start_camera()
except:
    camera.close()

def find_qr_area(frame):
    # Run a function that looks for codes in that frame
    decodedObjects = pyzbar.decode(frame) 
    
    # If the code finds more than one code...
    if len(decodedObjects) > 0:
        
        # For each QR code...
        for obj in decodedObjects: 
            width = obj.rect[2]
            heigt = obj.rect[3]
            area_of_qr = obj.rect[2]*obj.rect[3]
            return area_of_qr
 
def calc_distance(area_detected,area_of_object):
    #cant see beyond 10 inches and below 3 inches
    
    #when you are below 4 inches the area is too big
    #for the QR code to be seen properly
    
    #when you are too far away 
    print(area_detected)

    if area_detected:
        k = 0.96 #some constant which we can vary to get better results
        distance = ( k*(area_of_object*width*height)/area_detected )**(1/2)
    else:
        distance = -1
    return distance  

Starting PiCamera
Closing PiCamera


AttributeError: 'Camera' object has no attribute 'stream'

In [ ]:
length_of_qr = 3
area_of_qr = length_of_qr**2


try: 
    while True:
        time_start = time.time()
        area = find_qr_area(camera.capture())
        distance = calc_distance(area,area_of_qr)
        print(distance)
        
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert it to gray
#         ret,black_white = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)
#         IPython.display.display(PIL.Image.fromarray(black_white))
        
        print("FPS")
        print(1/(time.time()-time_start))
        
        IPython.display.clear_output(wait=True) 

finally:
    camera.close()
    print("done")

In [ ]:
#here are some measurements you can make
#using a rule and printing the area of pixels

'''
8747 pix area @ 4 inches
5329 pix area @ 5 inches
3721 pix area @ 6 inches
2652 pix area @ 7 inches
2025 pix area @ 8 inches
1600 pix area @ 9 inches
'''

#area of pixels
area_list = [8747,5329,3721,2652,2025,1600]

#the distance in inches measured
distance_list = [4,5,6,7,8,9]


degree = 2
                     #( x, y, the order of the polynomials)
weights = np.polyfit(distance_list,area_list, degree)
print(weights)

#this is the equation where
#x/input is distance and
#y/output is the area
model = np.poly1d(weights)
print(model)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt


plt.plot(distance_list,area_list,'ro')
plt.ylabel('distance')
plt.xlabel('area')
plt.show()

#now lets try to make a prediction
#if the distance is 3.5 inches

d =  9
area_pred = weights[2]*d**-2 - weights[1]*d + weights[0] 
print(area_pred)
